# Trasformatore xml2csv

In [10]:
# Import
import pandas as pd
import numpy as np
import matplotlib
import logging
import urllib
import io
import csv
import time
import os
from lxml import etree

DIRECTORY='../data/appalti_xml/'

In [45]:
def get_node(node):
    try:
        return node.text
    except (AttributeError, IndexError):
        return np.nan

def get_node2(node,index):
    try:
        return node[index].text
    except (AttributeError, IndexError):
        return np.nan

def get_date(node):
    try:
        data_inizio= node[0].text
    except (AttributeError, IndexError):
        data_inizio= np.nan

    try:
        data_fine= node[1].text
    except (AttributeError, IndexError):
        data_fine= np.nan
    
    return data_inizio, data_fine

def appalti_xml2csv(doc, id_comunicazione):
    csv_file = open(("../data/appalti_csv/%s.csv" % id_comunicazione), 'w')
    csv_writer = csv.writer(csv_file, delimiter=';', quoting=csv.QUOTE_ALL)
    root = doc.getroot()
    anno_rif = get_node(root[0][5])
    id_lotto=0
    rid=0
    arid=0

    lines=[]
    for lotto in root[1]:
        id_comunicazione_lotto = '-'.join([id_comunicazione,str(id_lotto)])
        cig = get_node(lotto[0])
        codice_fiscale_prop, denominazione  = get_node(lotto[1][0]), get_node(lotto[1][1])
        oggetto, scelta_contraente = get_node(lotto[2]), get_node(lotto[3])
        importo_aggiudicazione, importo_somme_liquidate = get_node(lotto[6]), get_node(lotto[8])
        data_inizio, data_fine = get_date(lotto[7]) 
        
        
        lines.append([id_comunicazione_lotto, anno_rif,cig, codice_fiscale_prop, denominazione, oggetto, scelta_contraente, importo_aggiudicazione, importo_somme_liquidate, data_inizio, data_fine])
        
        partecipanti, aggiundicatari = lotto[4], lotto[5]
        partecipanti.set('id_comunicazione_lotto', "%s"%id_comunicazione_lotto)
        aggiundicatari.set('id_comunicazione_lotto', "%s"%id_comunicazione_lotto)
        id_lotto+=1
        
    csv_writer.writerows(lines)
    csv_file.close()
    del lines
    
    
    
    for element in root.iter("raggruppamento","aggiudicatarioRaggruppamento"):
        if element.tag=='raggruppamento':
            element.set('rid', "%s"%'-'.join([id_comunicazione,str(rid)]))
            rid+=1
        elif element.tag=='aggiudicatarioRaggruppamento':
            element.set('arid', "%s"%'-'.join([id_comunicazione,str(arid)]))
            arid+=1
        else:
            raise NameError("Ops! Iterando su raggruppamenti ho trovato un errore.")

    csv_file = open(("../data/oe_csv/%s.csv" % id_comunicazione), 'w')
    csv_writer = csv.writer(csv_file, delimiter=';', quoting=csv.QUOTE_ALL)
    
    lines=[]
    for element in root.iter("membro", "partecipante","aggiudicatario"):
        parent=element.getparent()
        granparent = parent.getparent()
            
        if parent.tag=='raggruppamento':
            #linea è tid:granparent, rid:parent_rid, arid=none ... il resto uguale
            lines.append([
                granparent.attrib['id_comunicazione_lotto'],
                parent.attrib['rid'],
                np.NaN, 
                get_node(element.find("codiceFiscale")),
                get_node(element.find("ragioneSociale")),
                get_node(element.find("ruolo")),
                get_node(element.find("identificativoFiscaleEstero")),
                0, 
                0])
        elif parent.tag=='aggiudicatarioRaggruppamento':
            #identico al precedente ma speculare su rid e arid
            lines.append(
                [granparent.attrib['id_comunicazione_lotto'],
                np.NaN,
                parent.attrib['arid'],
                get_node(element.find("codiceFiscale")),
                get_node(element.find("ragioneSociale")),
                get_node(element.find("ruolo")),
                get_node(element.find("identificativoFiscaleEstero")),
                0,
                0])
        elif element.tag == "partecipante":
            # tid viene preso dal parent, rid e arid settati a Nan                                                                                                                 
            lines.append([
                parent.attrib['id_comunicazione_lotto'], 
                np.NaN,
                np.NaN, 
                get_node(element.find("codiceFiscale")),
                get_node(element.find("ragioneSociale")),
                get_node(element.find("ruolo")),
                get_node(element.find("identificativoFiscaleEstero")),
                1, 
                0])
        elif element.tag == "aggiudicatario":
            # tid viene preso dal parent, rid e arid settati a Nan                                                                                                                 
            lines.append([
                parent.attrib['id_comunicazione_lotto'], 
                np.NaN,
                np.NaN,
                get_node(element.find("codiceFiscale")),
                get_node(element.find("ragioneSociale")),
                get_node(element.find("ruolo")),
                get_node(element.find("identificativoFiscaleEstero")),
                0,
                1])
        else:
            raise NameError("Ops! Iterando su raggruppamenti ho trovato un errore.")
    
    csv_writer.writerows(lines)
    del lines
    csv_file.close()


In [46]:
# Per ogni xml in appalti_xml
    # Carica xml e trasforma

In [47]:
i = 0
for filename in os.listdir(DIRECTORY):
    if filename.endswith(".xml"): 
        id_comunicazione = filename.split('.')[0]
        doc = etree.parse(os.path.join(DIRECTORY, filename))
        
        appalti_xml2csv(doc, id_comunicazione)

1
1022
1022
1022
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1030
1035
1042
1042
1042
1042
1042
1042
1042
1042
1042
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1084
1096
1120
1120
1120
1120
1120
1120
1120
1120
1120
1120
1120
1120
1120
1141
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1154
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1159
1159
1159
1159
1161
1161
1161
1161
1161
1161
1161
1161
1161
1161
1161
1161
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
1165
116

KeyboardInterrupt: 